<a href="https://colab.research.google.com/github/Shreyas19-ai/cric_pred/blob/main/t20_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
t20_df = pd.read_csv('/content/ball_by_ball_it20.csv.zip')

In [ ]:
t24_df = pd.read_csv('/content/ball_by_ball_it20.csv.zip')

In [ ]:
t20_df

In [ ]:
t20_df.info()

In [ ]:
t20_df.describe()

In [ ]:
t20_df.isnull().sum()

In [ ]:
t20_df['Player Out'].unique()

In [ ]:
t20_df['Player Out'].value_counts()

In [ ]:
t20_df['Player Out'] = t20_df['Player Out'].fillna(0)

In [ ]:
t20_df.isnull().sum()

In [ ]:
t20_df['Method'].unique()

In [ ]:
t20_df['Player Out'].unique()

In [ ]:
t20_df.shape

In [ ]:
t20_df = t20_df.drop(columns=['Player Out Balls Faced','Player Out Runs','Runs to Get','Method'])

In [ ]:
t20_df.shape

In [ ]:
t20_df.info()

In [ ]:
df = t20_df[['Match ID','Date','Venue','Bat First','Bat Second','Ball','Over','Innings','Batter','Bowler','Runs From Ball','Wicket','Innings Runs','Innings Wickets','Target Score','Balls Remaining','Player Out']]

In [ ]:
df

In [ ]:
df = df.rename(columns={'Bat First':'Batting Team','Bat Second':'Bowling Team'})

In [ ]:
df

In [ ]:
df.sample(12)

In [ ]:
df['Innings Wickets'] = 10 - df['Innings Wickets']
df.rename(columns={'Innings Wickets':'Wickets Left'}, inplace=True)

In [ ]:
df

In [ ]:
df['Run Rate'] = df['Innings Runs']/df['Over']

In [ ]:
df

In [ ]:
df.rename(columns={'Innings Runs':'Current Score'}, inplace=True)
df

In [ ]:
df['Batting Team'].unique()

In [ ]:
df['Batting Team'].value_counts().head(20)

In [ ]:
teams = ['Pakistan',
'India',
'New Zealand',
'South Africa',
'Sri Lanka',
'West Indies',
'England',
'Australia',
'Bangladesh',
'Afghanistan',
'Zimbabwe',
'Ireland',
'Netherlands'
,'Scotland']

In [ ]:
df = df[df['Batting Team'].isin(teams)]
df = df[df['Bowling Team'].isin(teams)]

In [ ]:
df

In [ ]:
import pickle
pickle.dump(df,open('output.pkl','wb'))

# **Feature Extraction**

In [ ]:
new_df = pickle.load(open('output.pkl','rb'))

In [ ]:
new_df

In [ ]:
new_df['Venue'].unique()

In [ ]:
new_df['Venue'].value_counts().head(50)

In [ ]:
new_df['Venue'].value_counts()[new_df['Venue'].value_counts()>600]

In [ ]:
stadiums = new_df['Venue'].value_counts()[new_df['Venue'].value_counts()>600].index.to_list()

In [ ]:
new_df = new_df[new_df['Venue'].isin(stadiums)]

In [ ]:
new_df.rename(columns={'Venue':'Stadiums'}, inplace=True)

In [ ]:
new_df

In [ ]:
new_df.info()

In [ ]:
import pandas as pd

# Convert Runs From Ball to numeric before grouping
new_df['Runs From Ball'] = pd.to_numeric(new_df['Runs From Ball'], errors='coerce')

groups = new_df.groupby('Match ID')
match_ids = new_df['Match ID'].unique()
last_five = []
for id in match_ids:
    # Select only the 'Runs From Ball' column before applying rolling sum
    group_data = groups.get_group(id)['Runs From Ball']

    # Apply rolling sum and extend the list
    last_five.extend(group_data.rolling(window=30).sum().values.tolist())

In [ ]:
last_five

In [ ]:
new_df

In [ ]:
new_df['last five'] = last_five

In [ ]:
new_df

In [ ]:
new_df = new_df[['Batting Team','Bowling Team','Stadiums','Current Score','Wickets Left','Balls Remaining','Run Rate','last five','Target Score']]

In [ ]:
new_df.rename(columns={'Current Score':'current_score',
                       'Batting Team':'batting_team',
                       'Bowling Team':'bowling_team',
                       'Wickets Left':'wickets_left',
                       'Balls Remaining':'balls_left',
                       'last five':'last_five',
                       'Stadiums':'Stadium',
                       'Run Rate':'crr',
                       'Target Score':'runs_x'}, inplace=True)

In [ ]:
new_df

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df.dropna(inplace=True)

In [ ]:
new_df.isnull().sum()

In [ ]:
new_df = new_df.sample(new_df.shape[0]) # to avoid bias

In [ ]:
new_df

In [ ]:
new_df.sample(5)

In [ ]:
new_df.shape

In [ ]:
X = new_df.drop(columns=['runs_x'])
y = new_df['runs_x']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error
!pip install xgboost
from xgboost import XGBRegressor

In [ ]:
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse_output=False,drop='first'),['batting_team','bowling_team','Stadium'])
]
,remainder='passthrough')

In [ ]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.1,max_depth=12,random_state=1))])

In [ ]:
pipe1 = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',RandomForestRegressor(n_estimators=1000,max_depth=12,random_state=1))])

In [ ]:
pipe1.fit(X_train,y_train)
y_pred1 = pipe1.predict(X_test)
print(r2_score(y_test,y_pred1))
print(mean_absolute_error(y_test,y_pred1))

In [ ]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv = cross_val_score(pipe,X,y,cv=5,scoring='r2')
print(cv.mean())

In [ ]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
teams

In [ ]:
stadiums